In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve

from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
    
from sklearn.model_selection import train_test_split

In [9]:
df=pd.read_csv('Leads X Education.csv')
df

,Prospect ID,Lead Number,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,...,Get updates on DM Content,Lead Profile,City,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,I agree to pay the amount through cheque,A free copy of Mastering The Interview,Last Notable Activity
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,API,Olark Chat,No,No,0,0.0,0,0.00,...,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Modified
1,2a272436-5132-4136-86fa-dcc88c88f482,660728,API,Organic Search,No,No,0,5.0,674,2.50,...,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Email Opened
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.00,...,No,Potential Lead,Mumbai,02.Medium,01.High,14.0,20.0,No,Yes,Email Opened
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.00,...,No,Select,Mumbai,02.Medium,01.High,13.0,17.0,No,No,Modified
4,3256f628-e534-4826-9d63-4a8b88782852,660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.00,...,No,Select,Mumbai,02.Medium,01.High,15.0,18.0,No,No,Modified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9235,19d6451e-fcd6-407c-b83b-48e1af805ea9,579564,Landing Page Submission,Direct Traffic,Yes,No,1,8.0,1845,2.67,...,No,Potential Lead,Mumbai,02.Medium,01.High,15.0,17.0,No,No,Email Marked Spam
9236,82a7005b-7196-4d56-95ce-a79f937a158d,579546,Landing Page Submission,Direct Traffic,No,No,0,2.0,238,2.00,...,No,Potential Lead,Mumbai,02.Medium,01.High,14.0,19.0,No,Yes,SMS Sent
9237,aac550fe-a586-452d-8d3c-f1b62c94e02c,579545,Landing Page Submission,Direct Traffic,Yes,No,0,2.0,199,2.00,...,No,Potential Lead,Mumbai,02.Medium,01.High,13.0,20.0,No,Yes,SMS Sent
9238,5330a7d1-2f2b-4df4-85d6-64ca2f6b95b9,579538,Landing Page Submission,Google,No,No,1,3.0,499,3.00,...,No,NaN,Other Metro Cities,02.Medium,02.Medium,15.0,16.0,No,No,SMS Sent


In [10]:
original_df_copy=df.copy() # for prospect id and lead number

In [11]:
df.columns=[x.lower() for x in df.columns]
df.columns=df.columns.str.replace(' ','_')

In [47]:
def df_preprocessing(df2):
    df2=df2.drop(['a_free_copy_of_mastering_the_interview','country','get_updates_on_dm_content',
              'i_agree_to_pay_the_amount_through_cheque','specialization',
             'update_me_on_supply_chain_content','what_matters_most_to_you_in_choosing_a_course',
              'x_education_forums'],axis='columns')
    
    df2=df2.drop(['asymmetrique_activity_index','asymmetrique_profile_index'
            ,'asymmetrique_activity_score','asymmetrique_profile_score'],axis='columns')
    
    cols_to_remove=[]
    cols_to_remove.append('lead_quality')
    
    df2['what_is_your_current_occupation']=df2.what_is_your_current_occupation.replace(np.nan,'Other')
    df2.city=df2.city.replace([np.nan,'Select'],'Other Cities')
    
    df2=df2.drop(cols_to_remove,axis='columns')
    
    df2=df2.dropna()
    
    df2=df2.drop(['prospect_id','lead_number'],axis='columns')
    
    df2=df2.replace(['No','Yes'],[0,1])
    
    test_converted_values_df=df2.copy()
    
    Y=df2.converted
    X=df2.drop(['converted'],axis='columns')
    
    numeric_columns=[x for x in X.select_dtypes(include=np.number).columns]
    cat_columns=[x for x in X.select_dtypes(include=np.object).columns]
    
    X_train,X_test,Y_train,Y_test=train_test_split(X,Y,train_size=0.8,random_state=10)

    cbc=CatBoostClassifier(iterations=40)  #102  od_wait= 40
    cbc.fit(X_train,Y_train,cat_features=cat_columns,silent=True)
    
    Y_predicted_test=cbc.predict(X_test)
    Y_predicted_train=cbc.predict(X_train)
    Y_predicted_total=cbc.predict(X)
    
    probs_train=cbc.predict_proba(X_train)
    probs_train=[x[1] for x in probs_train]

    probs_test=cbc.predict_proba(X_test)
    probs_test=[x[1] for x in probs_test]

    probs_total=cbc.predict_proba(X)
    probs_total=[x[1] for x in probs_total]    
    
    fpr, tpr, thresholds = roc_curve(Y_train,probs_train, drop_intermediate = False )
    
    optimal_idx = np.argmax(tpr - fpr)
    optimal_score = thresholds[optimal_idx]
    
    df2['Scores']=probs_total
    
    df2['Scores']=df2['Scores']*100
    
    df2=df2.drop(['converted'],axis='columns')
    
    return df2,optimal_score,test_converted_values_df
    
    


In [48]:
final_df,optimal_score,test_converted_values_df=df_preprocessing(df)

In [49]:
optimal_score

0.6840391258873281

In [50]:
final_df[final_df['Scores']>optimal_score].shape

(4308, 22)

In [51]:
test_converted_values_df[test_converted_values_df['converted']==0].shape

(3100, 22)

In [52]:
test_converted_values_df[test_converted_values_df['converted']==1].shape

(2566, 22)